In [ ]:
from with_rules_method import average_consumption_calc
import serial
import time
import pymongo
from datetime import date


myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["test1"]
mycol = mydb["sensor_data"]

ser=serial.Serial('COM9',9600)   
data =[]                       # empty list to store the data
for i in range(5):
    b = ser.readline()         # read a byte string
    string_n = b.decode()  # decode byte string into Unicode  
    string = string_n.rstrip() # remove \n and \r
    l=string.split("=")
    if(l[0]=="No object in sight voltage"):
        str1="No"
    else:
        str1="Yes"
    mydict={"value":l[1] , "date":date.today() , "ir":str1}
    time.sleep(0.1)            # wait (sleep) 0.1 seconds
ser.close()

mycol=mydb["sensor_data"]
myquery={"date":date.today() , "ir":"No"}
mydoc = mycol.find(myquery)

val=[]
for x in mycol.find({},{"date":date.today() , "ir":"No"}):
    val.append(x)
for x in val:
    sum+=float(x)

ans=average_consumption_calc(sum,5)
print(ans)


In [3]:
from owlready2 import *	
onto = get_ontology("file://C://Users//Aishwarya//Desktop//Python//abc.rdf")
onto.load()

with onto:
    class Person(Thing):pass
    class Sensors(Thing):pass
    class House(Thing):pass
    class Modules(Thing):pass
    class Devices(Thing):pass
    class Watt(Thing):pass
    class IRSensor(Sensors):pass
    class GSM(Modules):pass
    class LightBulb(Devices):pass
    class Rooms(Thing):pass
        
    class present_in(Person>>Rooms, FunctionalProperty):pass
    class absent_in(Person>>Rooms,FunctionalProperty):pass
    class switches_on(Person>>LightBulb,FunctionalProperty):pass
    class switches_off(Person>>LightBulb,FunctionalProperty):pass
    class consumes(LightBulb>>Watt,FunctionalProperty):pass
    class no_consumption(LightBulb>>Watt,FunctionalProperty):pass
    class detects_motion(GSM>>Person,FunctionalProperty):pass
    class no_detection(GSM>>Person,FunctionalProperty):pass
    class sends_message(GSM>>Person,FunctionalProperty):pass
    class consists_of(House>>Devices,FunctionalProperty):pass
    class consists_of(House>>Rooms,FunctionalProperty):pass
    class consists_of(House>>Sensors,FunctionalProperty):pass
    class total_consumption(LightBulb>>float,FunctionalProperty):pass
    class average_consumption(LightBulb>>float,FunctionalProperty):pass
    class running_time(LightBulb>>float,FunctionalProperty):pass
    #class watt_consumed(LightBulb>>float,FunctionalProperty):pass
    
    rule=Imp()
    rule.set_as_rule("""Person(?p), present_in(?p,?r) , switches_on(?p,?l) -> consumes(?l,?w)""")

    rule1=Imp()
    rule1.set_as_rule("""Person(?p), IRSensor(?i), present_in(?p,?r) -> detects_motion(?i,?p)""")

    rule2=Imp()
    rule2.set_as_rule("""Person(?p), IRSensor(?i), absent_in(?p,?r) -> no_detection(?i,?p)""")
    #
    rule3=Imp()
    rule3.set_as_rule("""no_detection(?i,?p) -> absent_in(?p,?r)""")#

    rule4=Imp()
    rule4.set_as_rule("""detects_motion(?i,?p) -> present_in(?p,?r)""")
    
    rule5=Imp()
    rule5.set_as_rule("""Person(?p), GSM(?g) , absent_in(?p,?r), consumes(?l,?w)-> sends_message(?g,?p)""")
    
    rule6=Imp()
    rule6.set_as_rule("""LightBulb(?l) , running_time(?l,?h) , total_consumption(?l,?tw) , divide(?r,?tw,?h)-> average_consumption(?l,?r)""")
 

def average_consumption_calc(tot_c,run_t):
    consump=LightBulb(total_consumption=tot_c , running_time=run_t)
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)
    return consump.average_consumption
#onto.save("abcd.owl","rdfxml")